In [1]:
import os
import cv2
import random
import scipy as sp
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
np.set_printoptions(suppress=True, precision=5)

# dl imports
import torch

## CARLA dataset

In [2]:
from config import GlobalConfig
from data import CARLA_Data

root_dir = '/home/surya/Downloads/transfuser-2022/data/demo/scenario1/'
config = GlobalConfig()
demo_set = CARLA_Data(root=root_dir, config=config, routeKey='route2', load_raw_lidar=True)
print(f"There are {len(demo_set)} samples in Demo dataset")

100%|██████████| 1/1 [00:00<00:00, 151.79it/s]
There are 95 samples in Demo dataset


Create pytorch style dataloaders

In [3]:
from torch.utils.data import DataLoader
dataloader_demo = DataLoader(demo_set, shuffle=False, batch_size=2, num_workers=4)

In [4]:
sample_data = next(iter(dataloader_demo))
print(f"sample data is of type {type(sample_data)} and has following keys")

for k,v in sample_data.items():
    print(k, list(v.shape))
    
del sample_data

sample data is of type <class 'dict'> and has following keys
rgb [2, 3, 160, 704]
bev [2, 160, 160]
depth [2, 160, 704]
semantic [2, 160, 704]
speed [2]
x_command [2]
y_command [2]
target_point [2, 2]
target_point_image [2, 1, 256, 256]
raw_lidar [2, 10000, 3]
num_raw_lidar_points [2]
lidar [2, 2, 256, 256]
label [2, 20, 7]
ego_waypoint [2, 4, 2]


## Load pretrained model

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

from model import LidarCenterNet
model = LidarCenterNet(config, device, config.backbone, image_architecture='regnety_032', 
                           lidar_architecture='regnety_032', use_velocity=False)
model.to(device);
model.config.debug = True

model.eval();
checkpt = torch.load('/home/surya/Downloads/transfuser-2022/model_ckpt/transfuser/transfuser_regnet032_seed1_39.pth', map_location=device)
model.load_state_dict(checkpt)

<All keys matched successfully>

## Helper functions

In [6]:
from utils import VEHICLE_TO_LIDAR_FWD, LIDAR_HEIGHT

def generate_lane_points(waypoints, lane_width = 1.0):
    # input waypoints are 2d coordinates of centerline (N,2)
    # this function generates left and right lane corners
    # by subtracting and adding half lane width. We convert
    # to 3D Lidar coordinates by placing points at ground level

    n_points = waypoints.shape[0]
    lane_points = np.zeros((n_points * 2 , 3))
    
    # vehicle to lidar frame
    lane_points[:n_points, 0] = waypoints[:,0] + VEHICLE_TO_LIDAR_FWD
    lane_points[n_points:, 0] = waypoints[:,0] + VEHICLE_TO_LIDAR_FWD
    
    # left and right lanes
    lane_points[:n_points,1] = waypoints[:,1] - (lane_width * 0.5)
    lane_points[n_points:,1] = waypoints[:,1] + (lane_width * 0.5)
    
    # fixed height
    lane_points[:,2] = -LIDAR_HEIGHT
    return lane_points

In [13]:
bev_to_lidar = np.array([
            [0, -(1/8.0), 32],
            [-(1/8.0), 0, 16],
            [0 , 0, 1]
])


def convert_to_3d_bboxes(boxes_2d):
    n_boxes = boxes_2d.shape[0]
    bbox_3d = np.zeros((n_boxes, 7))

    # xy position from bev pixels to metres
    homogenous_coordinates = np.hstack([boxes_2d[:, :2], np.ones((n_boxes, 1))])
    bbox_3d[:, :2] = (bev_to_lidar @ homogenous_coordinates.T).T[:, :2]
    bbox_3d[:, 3] = boxes_2d[:, 3] / 8  # length 
    bbox_3d[:, 4] = boxes_2d[:, 2] / 8  # width
    bbox_3d[:, 6] = boxes_2d[:, 4]      # yaw

    # hardcoding z values
    bbox_3d[:, 2] = -1.25
    bbox_3d[:, 5] = 2.5
    return bbox_3d

## Visualization class

In [14]:
PCD_CAM_VIEW = dict(
            up=dict(x=0, y=0, z=1),
            eye=dict(x=-0.9, y=0, z=0.2)
    )

PCD_SCENE=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False,),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1),
)

In [15]:
from utils import get_lidar3d_plots, get_image2d_plots
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class Visualizer:
    def __init__(self, model_name, fig_width=1000, fig_height=800, pred_box_color='orange', 
                 waypoints_color = 'red', scene=PCD_SCENE, cam_view=PCD_CAM_VIEW):
        self.model_name = model_name

        # Create a 2x1 figure, top row for point cloud data
        # bottom row for rgb image data
        self.fig = make_subplots(rows=2, cols=1,specs=[[{"type": "scatter3d"}], [{}]], row_heights=[0.65, 0.35],
                                horizontal_spacing=0.0, vertical_spacing = 0.0)
        self.fig.update_layout(template="plotly_dark", scene=scene, scene_camera = cam_view,
                height = fig_height, width = fig_width, autosize=False,
                title=f"END TO END AUTONOMOUS DRIVING {self.model_name}", title_x=0.5, title_y=0.95,
                margin=dict(r=0, b=0, l=0, t=0))            
        self.fig.update_xaxes(showticklabels=False, visible=False, row=2, col=1)
        self.fig.update_yaxes(showticklabels=False, visible=False, row=2, col=1)
        
        # set export image option
        self.fig.to_image(format="png", engine="kaleido")
        self.pred_color = pred_box_color
        self.waypoints_color = waypoints_color
    
    def clear_figure_data(self):
        self.fig.data = []
    
    def get_bbox_colors(self, bbox_corners):
        return [self.pred_color] * bbox_corners.shape[0] if bbox_corners is not None else None
        
    def plot_waypoints(self, waypoints):
        return go.Mesh3d(x=waypoints[:,0], y=waypoints[:,1], z=waypoints[:,2], 
                         opacity=0.4, color=self.waypoints_color, 
                         hoverinfo='skip',showlegend=False)

    def add_lidar_plots(self, points, waypoints, pred_corners=None):
        lidar_3d_plots = get_lidar3d_plots(points, pc_kwargs=dict(colorscale='viridis', marker_size=1.2),
                                   pred_box_corners = pred_corners, 
                                   pred_box_colors = self.get_bbox_colors(pred_corners))
        lidar_3d_plots.append(self.plot_waypoints(waypoints))
        for trace in lidar_3d_plots:
            self.fig.add_trace(trace, row=1, col=1)

    def add_image_plots(self, image):
        self.fig.add_trace(get_image2d_plots(rgb_image = image), row=2, col=1)
            
    def visualize_predictions(self, points, waypoints, image, pred_corners=None):
        # clear previous data and plot lidar, image data
        self.clear_figure_data()
        self.add_lidar_plots(points=points, waypoints=waypoints, pred_corners=pred_corners)
        self.add_image_plots(image=image)
    
    def show_figure(self):
        self.fig.show()
        
    def save_to_png(self, output_path):
        self.fig.write_image(output_path)

## Demo video

In [16]:
%matplotlib agg
visualizer = Visualizer(model_name='TRANSFUSER')

from utils import boxes_to_corners_3d

frameIdx = 0
for data in tqdm(dataloader_demo):

    # load data to device, according to type
    for k in ['rgb', 'depth', 'lidar', 'label', 'ego_waypoint', \
              'target_point', 'target_point_image', 'speed']:
        data[k] = data[k].to(device, torch.float32)
    for k in ['semantic', 'bev']:
        data[k] = data[k].to(device, torch.long)
    
    # get model predictions
    _, outputs = model(data)

    # iterate through each sample in batch 
    bs = data['rgb'].shape[0]
    for i in range(bs):
        # extract input data
        rgb_image = data['rgb'][i].permute(1, 2, 0).detach().cpu().numpy().astype(np.uint8)
        tgt_waypoints = data['ego_waypoint'][i].detach().cpu().numpy()
        
        lidar_pc = data['raw_lidar'][i].detach().cpu().numpy()
        num_points = data['num_raw_lidar_points'].detach().cpu().numpy()[i]
        lidar_pc = lidar_pc[:num_points, :]

        # extract model predictions
        pred_waypoints = outputs['pred_wp'][i]
        pred_lanepoints = generate_lane_points(pred_waypoints)
        
        pred_boxes = outputs['detections'][i]
        pred_3d_boxes = convert_to_3d_bboxes(pred_boxes)
        pred_corners = boxes_to_corners_3d(pred_3d_boxes)
        break
    break
#         # plot all data
#         visualizer.visualize_predictions(lidar_pc, waypoints=pred_lanepoints, image=rgb_image)

#         # save figure
#         visualizer.save_to_png(f"testFolder/Frame{frameIdx}.png")
#         frameIdx +=1

  0%|          | 0/48 [00:00<?, ?it/s]/home/surya/miniconda3/envs/tfuse/lib/python3.7/site-packages/mmdet/models/utils/gaussian_target.py:227: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').

/home/surya/miniconda3/envs/tfuse/lib/python3.7/site-packages/mmdet/models/utils/gaussian_target.py:229: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'

In [17]:
visualizer.visualize_predictions(lidar_pc, waypoints=pred_lanepoints, image=rgb_image, pred_corners=pred_corners)

In [18]:
visualizer.show_figure()

In [ ]:
def convert_images_to_video(images_dir, output_video_path, fps : int = 20):
    input_images = [os.path.join(images_dir, *[x]) for x in sorted(os.listdir(images_dir)) if x.endswith('png')]
    
    if(len(input_images) > 0):
        sample_image = cv2.imread(input_images[0])
        height, width, _ = sample_image.shape
        
        # handles for input output videos
        output_handle = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

        # create progress bar
        num_frames = int(len(input_images))
        pbar = tqdm(total = num_frames, position=0, leave=True)

        for i in tqdm(range(num_frames), position=0, leave=True):
            frame = cv2.imread(input_images[i])
            output_handle.write(frame)
            pbar.update(1)

        # release the output video handler
        output_handle.release()
                
    else:
        pass

In [ ]:
convert_images_to_video('testFolder/', 'scenario1_route2.mp4')